In [ ]:
def train():
    
    for e in range(epi):
        done = False
        total_rewards = 0
        # State: Dict[Server, List[Task]] where Server is array format and Task is array format
        while not done:
            if env.get_stage() is EnvStage.Auction:
                auction_task = env.get_auction_obs()
                server_state = {server: task_pricing_agents[server].get_obs(auction_task, tasks, server)
                              for server, tasks in state.items()}
                server_prices =  {server: task_pricing_agents[server].price(server_state[server])
                                  for server in state.keys()}
                
                next_state, reward, done, info = env.auction_step(server_prices)
                
                total_rewards += reward
            elif env.get_stage() is EnvStage.ResourceAllocation:
                server_tasks = env.get_server_tasks()
                server_task_weighting = {
                    server: {
                        task: resource_weighting_agents[server].weight(server, task, [_task for _task in tasks if _task is not task])
                        for task in tasks
                    } for server, tasks in server_tasks.items()
                }
                
                next_state, rewards, done, info = env.resource_allocation_step(server_task_weighting)
                for server, tasks in next_state:
                    for task in tasks:
                        resource_weighting_agents[server].update_trajectory(server, task, [_task for _task in tasks if _task is not task], rewards[task])
                        